## RA2CE Feature: adaptation measures

This notebook explains how users can create different adaptation options that will influence the calculated damages and losses on roads. It also performs a Cost-Benefit Analysis (CBA) for each of the adaptation option to explore the cost effectiveness of these options.

In [ ]:
from pathlib import Path
from ra2ce.network.network_config_data.enums.road_type_enum import RoadTypeEnum
from ra2ce.ra2ce_handler import Ra2ceHandler
from ra2ce.network.network_config_data.enums.source_enum import SourceEnum
from ra2ce.analysis.analysis_config_data.analysis_config_data import (
    AnalysisConfigData,
    AnalysisSectionAdaptation,
    AnalysisSectionAdaptationOption,
    AnalysisSectionDamages,
)
from ra2ce.analysis.analysis_config_data.enums.analysis_damages_enum import (
    AnalysisDamagesEnum,
)
from ra2ce.analysis.analysis_config_data.enums.analysis_enum import AnalysisEnum
from ra2ce.analysis.analysis_config_data.enums.analysis_losses_enum import (
    AnalysisLossesEnum,
)
from ra2ce.analysis.analysis_config_data.enums.damage_curve_enum import DamageCurveEnum
from ra2ce.analysis.analysis_config_data.enums.event_type_enum import EventTypeEnum

from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.network_config_data import (
    HazardSection,
    NetworkConfigData,
    NetworkSection,
)


In [ ]:
root_dir = Path("data", "adaptation")

static_path = root_dir.joinpath("static")
hazard_path =static_path.joinpath("hazard")
network_path = static_path.joinpath("network")
output_path=root_dir.joinpath("output")

input_path = root_dir.joinpath("input") # path of the data files for all adaptation options and reference option

### Network config

The network must first be configured and overlaid with a hazard map. The current workflow only supports the following configurations:

- AggregateWlENum: only MEAN 
- SourceEnum must be set to SHAPEFILE. This is because we are running Losses which requires information about the traffic intensities, 
- Adaptation is for now only event-based for a single hazard map. Therefore the name of the hazard map should not start with "RP" and should start the risk-based damage workflow will be triggered. The hazard name should start with a letter.

In [ ]:
_network_section = NetworkSection(
    source= SourceEnum.OSM_DOWNLOAD,
    polygon= network_path.joinpath("polygon_jacksonvill_beach.shp"),
    road_types=[      
        RoadTypeEnum.SECONDARY,
        RoadTypeEnum.SECONDARY_LINK,
        RoadTypeEnum.PRIMARY,
        RoadTypeEnum.PRIMARY_LINK,
        RoadTypeEnum.TRUNK,
        RoadTypeEnum.MOTORWAY,
        RoadTypeEnum.MOTORWAY_LINK,
        RoadTypeEnum.RESIDENTIAL,
        RoadTypeEnum.UNCLASSIFIED,
    ],
    save_gpkg=True
)

_hazard = HazardSection(
    hazard_map=[Path(file) for file in hazard_path.glob("*.tif")],
    hazard_field_name= ["waterdepth"],
    aggregate_wl = AggregateWlEnum.MEAN,
    hazard_crs = "EPSG:4326",
)


_network_config_data = NetworkConfigData(
    root_path=root_dir,
    static_path=static_path,
    network=_network_section,
    hazard=_hazard
    )

In [ ]:
handler = Ra2ceHandler.from_config(_network_config_data, None)

handler.configure()

## Damages configuration

The configuration for the damages analysis must be defined here. The general configurations of damages are shared for all adaptation options!


### Damages:

- Since we are simulating the adaptation effects by modifying the hazard curves, only the Manual damage curve type is allowed: `DamageCurveEnum.MAN`
- `EventTypeEnum.EVENT` is mandatory since we are dealing with an event-base adaptation.

In [ ]:
_damages_section = AnalysisSectionDamages(
        analysis=AnalysisDamagesEnum.DAMAGES,
        event_type=EventTypeEnum.EVENT,
        damage_curve=DamageCurveEnum.MAN,
        save_gpkg=True,
        save_csv=True,
    )

## Adaptation options:

The adaptation options can now be defined. 

1. The `AnalysisSectionAdaptation` must be filled with general inputs applicable to all adaptation options:
    - discount rate: to account for inflation
    - initial_frequency: this the frequency of occurence of the considered hazard map at year 0
    - climate factor: accounting for the increase of likelihood over time fo the hazard map
    - time horizon: numbers of years for the CBA
2. A collection of adaptation options is to be specified:
    - Each adaptation option has an `id` which must match the input data structure in order to assign the input files correctly
    - The first adaptation option represents the initial situation (Business As Usual) and only requires a `name` and `id`.
    - The following adaptation options have extra required attributes to calculate the cost: `construction_cost`, `construction_interval`, 'maintenance_cost', 'maintenance_interval'

In [ ]:

# - adaptation
_adaptation_options = [
    AnalysisSectionAdaptationOption(
            id="AO0",
            name="No adaptation",
        ),
        AnalysisSectionAdaptationOption(
            id="AO1",
            name="Cheap construction, expensive maintenance",
            construction_cost=1000.0,  # this is the cost per meter
            construction_interval=20.0,
            maintenance_cost=0.0,  # this is cost per meter per year
            maintenance_interval=0.0,
        ),
]

_adaptation_section = AnalysisSectionAdaptation(
        analysis=AnalysisEnum.ADAPTATION,
        adaptation_options=_adaptation_options,
        discount_rate=0.025,  # correcting inflation 0.025 = 2.5%
        initial_frequency=0.05,  # yearly frequency of occurrence of the event (hazard map)
        climate_factor=0.007, # factor to correct for the positive increase of the frequency of occurrence of the event
        time_horizon=20,  # time horizon in years for the CBA analysis
        save_gpkg=True,
        save_csv=True,
    )

_analysis_config_data = AnalysisConfigData(
        input_path=input_path,
        static_path=static_path,
        output_path=output_path,
        analyses=[
            _damages_section,
            _adaptation_section,
        ],
        aggregate_wl=AggregateWlEnum.MEAN,
    )


For this example. There is 1 adaptation option defined `AO1`, in addition to the reference case 'AO0'. The corresponding files structure should then be the following (assuming MultiLinkLosses and damages based on all road types)

- root_dir/
  - input/
    - A00/
      - damages/
        - input/
          - damages_function/
            - all_road_types/
              - hazard_severity_damage_fraction.csv
              - max_damage_road_types.csv
    - A01/
  - static/
    - hazard/
    - network/
    - output_graph/


In [ ]:
_network_config_data.network.reuse_network_output = True
handler = Ra2ceHandler.from_config(_network_config_data, _analysis_config_data)

handler.configure()
handler.run_analysis()

In [ ]:
import geopandas as gpd
output_gdf = gpd.read_file(output_path.joinpath("adaptation.gpkg"))
output_gdf.head()

In [ ]:
output_gdf.explore("AO1_bc_ratio")